In [1]:
# 실습1
# 뉴스의 링크를 넘기면 해당 뉴스의 댓글 통계 정보를 딕셔너리로 반환하는 함수 만들기

# {'남자' : '68%', '여자' : '32%', '10대' : '1%', '20대' : '4%', '30대' : '17%',
#  '40대' : '45%', '50대' : '26%', '60대' : '8%' }

# 문제를 나누기
# 뉴스의 링크를 넘긴다
# 해당뉴스의 댓글 통계 정보를 추출해서 딕셔너리로 받는다.
# joson으로 저장한다.

In [2]:
#1. 일단 페이지 1개를 설정
# 이페이지가 정적인지 동적인지 알수 없다.
# 정적 : request, 동적: seliumn을 씀
# request가 더 빠름.
# 동적이면 안뜨것지
# 우선 request로 테스트!

import requests

## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

r = requests.get('https://n.news.naver.com/article/087/0000995105?ntype=RANKING', headers=headers)

test_news_page=r.text

from bs4 import BeautifulSoup
soup = BeautifulSoup(test_news_page, 'html.parser')
# soup으로 r page parsing 되서 옴

In [3]:
# {'남자' : '68%', '여자' : '32%', '10대' : '1%', '20대' : '4%', '30대' : '17%',
#  '40대' : '45%', '50대' : '26%', '60대' : '8%' }
test=soup.select(".u_cbox_chart_cont")
test #[] : request로 가져왔을때 이러면 이부분은 동적
# 페이지는 정적부분 + 동적 부분으로 이루어 지기 때문에
# 즉 여기서 이걸 가져오려면 request+BequtifulSoup으로 안됨.

[]

In [4]:
# selenuim을 사용해보자!
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless')
service = ChromeService(executable_path='C:/Users/admin/psm/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

driver.get('https://n.news.naver.com/article/087/0000995105?ntype=RANKING')
time.sleep(10)
chartDiv=driver.find_element(By.CLASS_NAME,'u_cbox_chart_cont')

In [5]:
innerChart=chartDiv.find_element(By.CLASS_NAME,'u_cbox_chart_cont_inner')
perNum=innerChart.find_elements(By.CLASS_NAME,'u_cbox_chart_per')

# innerChart.text #'76%\n남자\n24%\n여자\n0%\n10대\n5%\n20대\n27%\n30대\n46%\n40대\n17%\n50대\n4%\n60대
names=innerChart.find_elements(By.CLASS_NAME,'u_cbox_chart_cnt')


# %들의 class 이름이 u_cbox_chart_per 임.
# name들의 class 이름이 u_cbox_chart_cnt 임.

In [6]:
innerChart.text
names[3].text

'20대'

In [7]:
# # 그럼 이페이지의 class=u_cbox_chart_per, class=u_cbox_chart_cnt 를 dic 하나에 모아서 저장되는지 시도!
# result_dict = {}

# # 요소를 반복하면서 데이터를 딕셔너리에 저장
# for i in range(min(len(perNum), len(names))):
#     key = names[i].text
#     value = perNum[i].text
#     result_dict[key] = value
    

In [8]:
# print(result_dict)

In [9]:
# 여기까지를 함수로 만들기
def article_value(dirver,href):
    chartDiv=driver.find_element(By.CLASS_NAME,'u_cbox_chart_cont')
    innerChart=chartDiv.find_element(By.CLASS_NAME,'u_cbox_chart_cont_inner')
    perNum=innerChart.find_elements(By.CLASS_NAME,'u_cbox_chart_per')
    names=innerChart.find_elements(By.CLASS_NAME,'u_cbox_chart_cnt')
    
    result_dict = {}
    
    for i in range(min(len(perNum), len(names))):
        key = names[i].text
        value = perNum[i].text
        result_dict[key] = value
        
    return result_dict

In [ ]:
# 이제 이걸 페이지 마다 하면 됨. 근데 모든 페이지마다 우선 하면 힘드니까 페이지도 하나하나 짜르자
# 1단계 00언론사의 랭킹 페이지(20개 기사)의 댓글만 가져와서 저장

#  00 언론사의 기사 링크가 있는 걸 다 가져올수 있는지 테스트

# 1. 00 언론사의 기사 랭킹페이지는 정적인가 동적인가 판단
# request로 때린다.

import requests

## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

r = requests.get('https://media.naver.com/press/028/ranking?type=popular', headers=headers)

test_rankingnews_page=r.text

from bs4 import BeautifulSoup
soup = BeautifulSoup(test_rankingnews_page, 'html.parser')
# soup으로 r page parsing 되서 옴
thumb_list=soup.select(".as_thumb")

# 아? 위는 안되고 아래는 되는 이유? thumb_list가 ResultSet 객체라서 라는데 이해 안됨.
# a_tags = thumb_list.selects('a')
a_tags = [thumb.select_one('a') for thumb in thumb_list]

Id_dic={}

for a_tag in a_tags:
    href = a_tag.get('href')
    
    article_value(href)
    
    split=href.split('?')
    split=split[0].split('/')
    Id=split[-1]
    
    for i in range(len(result_dict)):
        key = Id
        value = result_dict
        Id_dic[key] = value
        
    

In [ ]:
print(Id_dic)

In [ ]:
# 여기까지를 하나의 함수로 만들기
def ranking_news(driver, href):
#     thumb_list=driver.select(".as_thumb")
    thumb_list = driver.find_elements(By.CLASS_NAME, "as_thumb")

    a_tags = [thumb.select_one('a') for thumb in thumb_list]

    Id_dic={}
    
    for a_tag in a_tags:
#         href = a_tag.get('href')

        article_value(driver, href)  # 함수 호출

#         article_value(href) #함수 호출

        split=href.split('?')
        split=split[0].split('/')
        Id=split[-1]

        for i in range(len(result_dict)):
            key = Id
            value = result_dict
            Id_dic[key] = value
    
    
    

In [ ]:
print(Id_dic)

In [ ]:
import requests

## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

r = requests.get('https://news.naver.com/main/ranking/popularDay.naver', headers=headers)

test_rankingnews_page=r.text

from bs4 import BeautifulSoup
soup = BeautifulSoup(test_rankingnews_page, 'html.parser')
# soup으로 r page parsing 되서 옴
box_list=soup.select(".rankingnews_box")

# 각 요소에서 'a' 태그 선택 : 사이트 링크
a_tags = [box.select_one('a') for box in box_list]

#key로 들어갈 언론사 이름 추출
name_key = [a.select_one('.rankingnews_name').text for a in a_tags]

company_dic={}

for a_tage in a_tags:
    href = a_tag.get('href')
    
    ranking_news(driver, href)
    
    for i in range(len(result_dict)):
        key = Id
        value = result_dict
        company_dic[key] = value


In [ ]:
print(company_dic)

In [ ]:
# 2단계 랭킹페이지에서 페이지들의 링크로 모든 언론사 저장.

In [ ]:
# 2. 페이지 하나에서 댓글을 추출할수 있는지 경로 확인

# 3. json 저장(test)

# 4. 방법을 동일하게 해서 반복을 돌린다!

In [ ]:
# 실습2
# SBS, MBC, KBS 세 언론사의 모든 랭킹뉴스 통계 정보를 json 파일로 저장해주세요

In [ ]:
# 실습3
# 각 뉴스의 댓글 수를 스크랩해서 저장해주세요.

In [ ]:
# 실습4
# 수집한 뉴스데이터를 이용해 다음 문제를 풀어주세요.
## 1. 20대가 가장 많이 본(댓글을 많이 작성한) 뉴스

## 2. 가장 댓글이 적은 뉴스의 댓글 수와 뉴스 링크, 뉴스 번호

## 3. 언론사 번호와 언론사 이름으로 구성된 데이터 프레임을 만들고 merge를 이용해 각 뉴스에 언론사 이름을 붙여주기

## 4. 각 언론사별 평균 댓글 수(댓글 수로 내림차순 정렬)

## 5. 여성의 댓글수가 가장 많은 언론사

## 6. 각 언론사 별 댓글을 많이 작성한 연령대 top3